In [19]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

mpl.style.use('ggplot')
plt.rcParams['savefig.dpi'] = 300 #图片像素
plt.rcParams['figure.dpi'] = 300 #分辨率
plt.rc('figure', figsize=(14, 10))#设置图片大小
sns.set_style('white')

%run Util.ipynb
%run Constant.ipynb
%run Policy_Loader.ipynb
%run BreakSectionManager.ipynb
%run Operation.ipynb
%run Evaluation.ipynb


# 沿着时间线，模拟买卖
class ProRocketRegression:
    
    def __init__(self,scale,start_date,end_date,policy_name):
        self.scale = scale
        self.start_date = start_date
        self.end_date = end_date
        self.policy_name = policy_name
        
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        self.account = Account(self.context,self.policy_name)
        
        self.operations={}
        
        
    def get_to_buy_candidate_stocks(self,cur_date,all_sections):
        already_bought = set(self.account.get_already_bought_symbols())
        
        part = all_sections[all_sections['d_s_date']==cur_date]
        
        if(part.shape[0]>0):
            stocks = part['symbol'].values
            
            for s in stocks:
                already_bought.add(s)
    
            if(self.debug==True):
                print(cur_date+' found to buy stocks '+ str(already_bought))
                
        return list(already_bought)
    
    
    def get_to_sell_candidate_stocks(self,cur_date):
        symbols = list(set(self.account.position.keys()))
        
        return symbols
        
    def get_all_sections(self):
        symbols = get_symbols(self.scale,home)
        
        all_sections = pd.DataFrame(columns=break_section_columns)
        for symbol in symbols:
            sectionManager = BreakSectionManager(symbol,self.context)
            section_df = sectionManager.get_all_break_sections()
            
            all_sections = pd.concat([section_df,all_sections], ignore_index=True, sort=False)
            
        return all_sections
    
    def get_operation(self,symbol):
        if(symbol in self.operations):
            return self.operations[symbol]
        else:
            operation = Operation(self.account,symbol,self.context,self.policy_name)
            self.operations[symbol] = operation
            
            return operation
        
    
    def try_to_buy(self,cur_date,candidates):
        if(candidates is None):
            return
        
        for symbol in candidates:
            operation = self.get_operation(symbol)
        
            if(operation.is_buy_point(cur_date)==True):
                operation.open_opsition(cur_date)


    def try_to_sell(self,cur_date,candidates):
        for symbol in candidates:
            operation = self.get_operation(symbol)
            
            if(operation.is_sell_point(cur_date)==True):
                if(self.debug==True):
                    print('try to sell '+ symbol)

                operation.sell_stock(cur_date)
            
    def build_asset_file_path(self):
        return report_dir+'multip_stocks_account_asset_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+operation_version+'.csv'
    
    def build_deal_file_path(self):
        return report_dir+'multip_stocks_deals_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+operation_version+'.csv'
    
    def persist(self):
        self.account.get_asset_df().to_csv(self.build_asset_file_path(),index=False)
        self.account.deal_df.to_csv(self.build_deal_file_path(),index=False)
        
    def draw_asset(self):
        if(self.policy_loader.enable_painter()==True):
            painter = AssetPainter()

            asset_file = self.build_asset_file_path()
            painter.draw_month_profit_bars(asset_file)
            painter.draw_audit_asset_bars(asset_file)

    def start_regression(self):
        start_datetime = to_datetime(self.start_date)
        end_datetime = to_datetime(self.end_date)
        
        all_sections = self.get_all_sections()
        market = Metric('SH#999999',self.context)
        
        if(self.debug==True):
            print('regression start ... ')
            
        while(start_datetime<=end_datetime):
            cur_date = start_datetime.strftime(YMD_format)
            
            if(market.is_today_open(cur_date)):
                self.account.daily_audit(cur_date)

                if(self.account.can_open_new_stock()==True):
                    candidates = self.get_to_buy_candidate_stocks(cur_date,all_sections)
                    self.try_to_buy(cur_date,candidates)
                    
                if(self.account.has_shares()==True):
                    candidates = self.get_to_sell_candidate_stocks(cur_date)
                    self.try_to_sell(cur_date,candidates)
                    
            start_datetime += dt.timedelta(days = 1)
            
            if(int(cur_date.split('-')[2])==28):
                print(cur_date,end=' ')
      
        self.persist()
        self.draw_asset()
        evaluation = Evaluation(self.account,self.context,self.policy_name)
        evaluation_df = evaluation.get_evaluation_report_of_multip_stocks(self.start_date,self.end_date)
        
        if(self.debug==True):
            print('regression done !')
            
        return evaluation_df



In [ ]:
scale = 60
start = '2000-02-01'
end = '2019-08-30'
policy_name='rocket_break_v6_enable_multip_stocks.json'

regression = ProRocketRegression(scale,start,end,policy_name)
evaluation_df = regression.start_regression()

evaluation_df.T

2000-02-28 2000-03-28 2000-04-28 2000-05-28 2000-06-28 2000-07-28 2000-08-28 2000-09-28 2000-10-28 2000-11-28 2000-12-28 2001-01-28 2001-02-28 2001-03-28 2001-04-28 2001-05-28 2001-06-28 2001-07-28 2001-08-28 2001-09-28 2001-10-28 2001-11-28 2001-12-28 2002-01-28 2002-02-28 2002-03-28 2002-04-28 2002-05-28 2002-06-28 2002-07-28 2002-08-28 2002-09-28 2002-10-28 2002-11-28 2002-12-28 2003-01-28 2003-02-28 2003-03-28 2003-04-28 2003-05-28 2003-06-28 2003-07-28 2003-08-28 2003-09-28 2003-10-28 2003-11-28 2003-12-28 2004-01-28 2004-02-28 2004-03-28 2004-04-28 2004-05-28 2004-06-28 2004-07-28 2004-08-28 2004-09-28 2004-10-28 2004-11-28 2004-12-28 2005-01-28 2005-02-28 2005-03-28 2005-04-28 2005-05-28 2005-06-28 2005-07-28 2005-08-28 2005-09-28 2005-10-28 2005-11-28 2005-12-28 2006-01-28 2006-02-28 2006-03-28 2006-04-28 2006-05-28 2006-06-28 2006-07-28 2006-08-28 2006-09-28 2006-10-28 2006-11-28 2006-12-28 2007-01-28 2007-02-28 2007-03-28 2007-04-28 2007-05-28 2007-06-28 2007-07-28 2007-08-28

In [ ]:
columns = ['policy_name','short','mid','in_hand_asset_drawdown','avg_drawdown','float_profit_drawdown','avg_year_return',
              'year_regular_return','mean_year_return','std_year_return','mean_month_return','std_month_return','profit_month_rate']
evaluation_df[columns]

In [ ]:
plt.rc('figure', figsize=(14,8))
regression.account.asset['assets'].plot(kind='bar')